# generate mapping from ICAO to ISO alpha-2 country code

Unfortunately, I couldn't find this mapping anywhere, but the ICAO country codes are used in the EUROCONTROL data.

The mapping is based on [this file](https://github.com/ip2location/ip2location-iata-icao/blob/master/iata-icao.csv). It's a bit nasty, since there are inconsistencies in the data.

In [13]:
import pandas as pd

df = pd.read_csv("iata-icao.csv")
df

,country_code,region_name,iata,icao,airport,latitude,longitude
0,AE,Abu Zaby,AAN,OMAL,Al Ain International Airport,24.2617,55.6092
1,AE,Abu Zaby,AUH,OMAA,Abu Dhabi International Airport,24.4330,54.6511
2,AE,Abu Zaby,AYM,NaN,Yas Island Seaplane Base,24.4670,54.6103
3,AE,Abu Zaby,AZI,OMAD,Al Bateen Executive Airport,24.4283,54.4581
4,AE,Abu Zaby,DHF,OMAM,Al Dhafra Air Base,24.2482,54.5477
...,...,...,...,...,...,...,...
9153,ZW,Masvingo,MVZ,FVMV,Masvingo Airport,-20.0553,30.8591
9154,ZW,Matabeleland North,HWN,FVWN,Hwange National Park Airport,-18.6299,27.0210
9155,ZW,Matabeleland North,VFA,FVFA,Victoria Falls Airport,-18.0959,25.8390
9156,ZW,Matabeleland North,WKI,FVWT,Hwange Town Airport,-18.3630,26.5198


In [14]:
# assume that the first two characters of the ICAO airport code are the same as the ICAO country code
df["icao_country"] = df["icao"].str[:2]
df

,country_code,region_name,iata,icao,airport,latitude,longitude,icao_country
0,AE,Abu Zaby,AAN,OMAL,Al Ain International Airport,24.2617,55.6092,OM
1,AE,Abu Zaby,AUH,OMAA,Abu Dhabi International Airport,24.4330,54.6511,OM
2,AE,Abu Zaby,AYM,NaN,Yas Island Seaplane Base,24.4670,54.6103,NaN
3,AE,Abu Zaby,AZI,OMAD,Al Bateen Executive Airport,24.4283,54.4581,OM
4,AE,Abu Zaby,DHF,OMAM,Al Dhafra Air Base,24.2482,54.5477,OM
...,...,...,...,...,...,...,...,...
9153,ZW,Masvingo,MVZ,FVMV,Masvingo Airport,-20.0553,30.8591,FV
9154,ZW,Matabeleland North,HWN,FVWN,Hwange National Park Airport,-18.6299,27.0210,FV
9155,ZW,Matabeleland North,VFA,FVFA,Victoria Falls Airport,-18.0959,25.8390,FV
9156,ZW,Matabeleland North,WKI,FVWT,Hwange Town Airport,-18.3630,26.5198,FV


In [15]:
# crap, looks like we have duplicates
df.dropna().groupby("country_code")["icao_country"].nunique().sort_values(
    ascending=False
)

country_code
US    33
AU    24
RU    11
CN    10
BR     7
      ..
GT     1
GU     1
GW     1
GY     1
ZW     1
Name: icao_country, Length: 229, dtype: int64

In [16]:
filtered_df = df.groupby("country_code").filter(
    lambda x: x["icao_country"].nunique() > 1
)
filtered_df

,country_code,region_name,iata,icao,airport,latitude,longitude,icao_country
210,AU,Australian Capital Territory,CBR,YSCB,Canberra Airport,-35.3069,149.1950,YS
211,AU,New South Wales,ABX,YMAY,Albury Airport,-36.0678,146.9580,YM
212,AU,New South Wales,ARM,YARM,Armidale Airport,-30.5281,151.6170,YA
213,AU,New South Wales,BEO,YLMQ,Lake Macquarie Airport (Belmont Airport),-33.0667,151.6480,YL
214,AU,New South Wales,BHQ,YBHI,Broken Hill Airport,-32.0014,141.4720,YB
...,...,...,...,...,...,...,...,...
9027,YE,San'a,SAH,OYSN,Sana'a International Airport,15.4764,44.2197,OY
9028,YE,San'a',RXA,NaN,Ar Rawdah Airport,14.4500,47.2830,NaN
9029,YE,Shabwah,AXK,OYAT,Ataq Airport,14.5513,46.8262,OY
9030,YE,Shabwah,BHN,OYBN,Beihan Airport,14.7820,45.7201,OY


In [17]:
df.dropna().groupby("icao_country")["country_code"].nunique().sort_values(
    ascending=False
)

icao_country
FM    5
TF    4
TN    4
LF    3
YP    3
     ..
KY    1
KX    1
KW    1
KV    1
ZY    1
Name: country_code, Length: 303, dtype: int64

In [18]:
df_clean = df.dropna()

# Find the most frequent country_code for each icao_country
most_frequent = (
    df_clean.groupby(["icao_country", "country_code"]).size().reset_index(name="count")
)
most_frequent = most_frequent.loc[
    most_frequent.groupby("icao_country")["count"].idxmax()
]

# Create the new DataFrame with one line per icao_country
result_df = most_frequent[["icao_country", "country_code"]]

# Reset the index of the result DataFrame
result_df = result_df.reset_index(drop=True)

print(result_df)

    icao_country country_code
0             AG           SB
1             AN           NR
2             AT           PG
3             AY           PG
4             BG           GL
..           ...          ...
298           ZP           CN
299           ZS           CN
300           ZU           CN
301           ZW           CN
302           ZY           CN

[303 rows x 2 columns]


In [19]:
result_df.dropna().groupby("icao_country")["country_code"].nunique().sort_values(
    ascending=False
)

icao_country
AG    1
SY    1
SV    1
SU    1
SS    1
     ..
KV    1
KU    1
KT    1
KS    1
ZY    1
Name: country_code, Length: 303, dtype: int64

In [20]:
result_df.to_csv("icao_country.csv", index=False)